The aim of this task was to find the shortest path from 12 to 34.

In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("yarn"))

In [2]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    prev_v, prev_path, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_path + [next_v])

def found():
    return  paths.filter(lambda x: x[0] == v_to).count()

v_from,v_to = 12,34
numPartitions = 4
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()

#here we bring data to the view follower user (k,v) for join bc we do join on k
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(numPartitions).persist()

#here we create an array for a path 
paths = sc.parallelize([(v_from, [v_from])]).partitionBy(numPartitions)

while found()==0:
    paths = paths.join(forward_edges, numPartitions).map(step)
paths.cache()
shortest_path = paths.filter(lambda x: x[0] == v_to).collect()

In [3]:
print (str(shortest_path[0][1]).strip('[]'))

12, 422, 53, 52, 107, 20, 23, 274, 34


In [4]:
sc.stop()